# 案例实战：电商用户对商品喜好的预测

在互联网行业中，要说哪一个领域是数据分析最发光发热的地方，那就是**电子商务领域**。绝大多数电商网站，都强依赖于数据分析帮助其挖掘新的用户与订单增长机会。

典型的应用场景如下：

- 通过数据分析发现不同类型用户对于商品、店铺的偏好，从而进行针对性的推荐；
- 通过数据分析发现用户未来潜在的购买概率，以决定是否要给用户发放红包或者优惠券来引导用户下单。

无论是哪个方向，数据分析都能显著提升业务的效果，是电商业务不可或缺的一环。下面通过一个真实的电商数据集进行分析来复习前面学习到的 Pandas 相关知识。

## 1 任务背景

阿普尔星球最大的电商网站阿普闪购计划策划一场推广活动，通过发短信的形式，向潜在的用户发送广告和优惠信息，吸引他们来阿普闪购注册并购物。由于预算以及短信服务商的限制，**没有办法对大范围的用户投放**，这就需要缩小人群的范围，找出最有可能产生转化的人群。此外，人们的下单行为往往也和时间呈现一定的相关性，什么时候推送营销短信也很重要。

作为数据分析师，你的目标是：

- 通过数据分析，找到最有可能转化的人群特征（年龄、性别、地域）等；
- 通过数据分析，决定出最有利于转化的营销短信投放时间。

通过和多个部门的沟通，最终你从阿普闪购的数据部门申请到了如下数据权限：

- 用户行为表 `user_behavior_time_resampled.csv`：最近 6 个月的用户行为记录。
- VIP 会员表 `vip_users.cs`：用户 VIP 会员开通情况。
- 用户信息表 `user_info.csv`：用户的相关信息。

表信息介绍：每一个表的第一行是表头，剩下的就是具体数据。每个表头的含义“字如其名”，很好理解。

`user_behavior_time_resampled.csv`：

![](user_behavior_time_resampled.png)

`vip_user.csv`：

![](vip_user.png)

`user_info.csv`：

![](user_info.png)

## 2 加载数据

用三个 DataFrame 去加载这三个 csv 文件。代码如下：

In [1]:
import pandas as pd

df_user_log = pd.read_csv("data_set/user_behavior_time_resampled.csv")
df_vip_user = pd.read_csv("data_set/vip_user.csv")
df_user_info = pd.read_csv("data_set/user_info.csv")

## 3 timestamp 的解析

目前 user_log 表中存在两个时间戳字段，比较迷惑。这两个字段究竟代表什么呢？两个时间戳字段 time_stamp 和 timestamp 的值是不一样的。

- time_stamp 看起来是个整数，值比较小;
- timestamp 看起来是个浮点数，值比较大，普遍在五位数;
- 其他暂时看不出什么有用的信息。

对于不明确含义的字段，通常的做法是先看一下它的边界（即最大值和最小值）。

In [2]:
time_stamp_max = str(df_user_log['time_stamp'].max())
time_stamp_min = str(df_user_log['time_stamp'].min())
print("time_stamp max: " + time_stamp_max, "time_stamp min: " + time_stamp_min)

timestamp_max = str(df_user_log['timestamp'].max())
timestamp_min = str(df_user_log['timestamp'].min())
print("timestamp max: " + timestamp_max, "timestamp min: " + timestamp_min)

time_stamp max: 1112 time_stamp min: 511
timestamp max: 86399.99327792758 timestamp min: 0.1078739773348047


从数据集的描述中，用户行为表是用户 6 个月的行为，那 time_stamp 最大 1112，最小 511 看起来就特别像日期。代表最小日期是 5 月 11 日，最大日期是 11 月 12 日。

那既然 time_stamp 是日期，那 timestamp 会不会是具体的时间呢？timestamp 的最大值为 86399 ，而一天最大的秒数为 24*3600 = 86400。两个数字非常接近，那基本可以认定 timestamp 代表的是一天中的第几秒发生了这个行为。

破解了两个时间字段的问题，为了避免后面有歧义，我们将 time_stamp 列重命名为 date。

In [3]:
df_user_log.rename(columns={"time_stamp": "date"}, inplace=True)
df_user_log

,user_id,item_id,cat_id,seller_id,brand_id,date,action_type,timestamp
0,328862,323294,833,2882,2661.0,829,click,62242.300959
1,328862,623866,1271,2882,2661.0,829,click,75598.841378
2,328862,239288,602,420,4953.0,801,click,69606.479645
3,328862,197239,602,420,4953.0,801,click,62758.912202
4,328862,524981,664,2382,1272.0,602,click,39651.123222
...,...,...,...,...,...,...,...,...
10985061,208016,869502,898,3716,5508.0,1110,click,58128.784724
10985062,208016,51712,898,3763,8434.0,1110,click,84907.294322
10985063,208016,952056,898,3763,8434.0,1110,click,64118.247166
10985064,208016,952056,898,3763,8434.0,1110,click,18877.391838


## 4 清洗数据

在现实世界中，除了字段的含义可能有不对之外，数据集本身也会有缺失的情况，比如某些条记录缺少字段之类的。所以在正式分析之前，我们都需要对这些缺失值进行处理。学过使用 isna 函数来获得缺失值，pandas 同时还提供了另一个类似的函数 isnull，这次我们来使用这个函数完成任务。

### 清洗 user_log 表

In [4]:
df_user_log.isnull().sum()

user_id            0
item_id            0
cat_id             0
seller_id          0
brand_id       18132
date               0
action_type        0
timestamp          0
dtype: int64

从上述结果中可以看出，user_log 表中大概有 1.8 w 条数据缺少品牌 id 的字段，缺失率为0.16%（1.8w/1098w)，一般这个数据量级不会影响到数据分布的分析，暂时不处理。

### 清洗 user_info 表

In [5]:
df_user_info.isnull().sum()

user_id         0
age_range    2217
gender       6436
dtype: int64

发现 user_info 表中有 2217 条记录缺失了年龄字段，有 6436 条记录中缺失了性别字段。我们会尝试分析目标用户群性别和年龄的分布，所以这里需要对这些缺失的值进行处理。由于缺失的数据量较少，所以我们选择**直接删掉**这些有缺失的记录。

In [9]:
df_user_info = df_user_info.dropna()
df_user_info

,user_id,age_range,gender
0,376517,6.0,1.0
1,234512,5.0,0.0
2,344532,5.0,0.0
3,186135,5.0,0.0
4,30230,5.0,0.0
...,...,...,...
424164,297789,0.0,1.0
424165,395814,3.0,1.0
424166,245950,0.0,1.0
424168,272535,6.0,1.0


user_info 的表记录数变为了 417708（之前是 424170，减少了 6462 条记录）。现在，我们通过上述方法排除了字段值为 NULL 的记录。

但是在上面对 user_info 表中年龄字段的描述中说到，age_range 为 0.0 或者 gender 为 0 同样代表数据缺失。我们来看一下这部分的数据有多少。

In [10]:
print(df_user_info.loc[df_user_info["age_range"] == 0.0, ["user_id"]].shape)
print(df_user_info.loc[df_user_info["gender"] == 0.0, ["user_id"]].shape)

(90638, 1)
(285634, 1)


第一行代表年龄为空的记录数，第二行代表性别为空的记录数。整体的比例并不低，但这里我们选择暂时保留这些数据，在后续分析环节再进行过滤。（因为当某条记录性别为空年龄不为空时，对于分析年龄分布仍然有价值，反之亦然）。

### 清洗 vip_user 表

In [11]:
df_vip_user.isnull().sum()

user_id        0
merchant_id    0
label          0
dtype: int64

从输出结果来看， vip_user 这个表没有数据缺失，不用清理。

## 5 数据分布分析

我们希望从数据分布中分析出最有可能转化的用户的特征。直白地说，我们希望分析出目前阿普闪购的用户中，**什么年龄段的用户最多，什么性别的用户最多**。进一步，希望分析出**下单**的用户里，**年龄和性别的分布**，这样才能知道给哪个年龄段和哪个性别的用户发营销短信是最有用的。

这里我们会用到一个非常强大的 pandas 函数：value_counts，value_counts 一般是针对 Series 对象，用于统计 Series 中有多少个不同的值，以及每个值出现的次数。比如：

### 用户年龄分布分析

In [12]:
# 我们先分析用户表中用户的年龄段分布。如上文所说，数据分布可以直接使用 DataFrame 的 value_counts 函数。
df_user_info.age_range.value_counts()

3.0    110952
0.0     90638
4.0     79649
2.0     52420
5.0     40601
6.0     35257
7.0      6924
8.0      1243
1.0        24
Name: age_range, dtype: int64

除开未知的数据不看（假设未知的部分的分布符合整体的分布），可以发现，除去取值为 0 （代表未知）之外，取值为 3.0 和 4.0 的占到绝大多数，回忆上面数据集的定义，取值为 3 代表 25～30 岁，取值为 4 代表 30～34 岁。所以年龄在可以得到， 25～34 岁之间的用户占绝大多数。（age_range 取值 3.0、4.0）。

然后，我们可以通过代码计算出 25～34 岁用户的比例。首先通过 loc 函数筛选出所有不等于 0 的记录，然后计算年龄段等于 3.0 与年龄段等于 4.0 的总数除以所有非 0 记录的总数，计算的代码如下：

In [20]:
user_ages = df_user_info.loc[df_user_info["age_range"] != 0, "age_range"]
user_ages.loc[(user_ages == 3) | (user_ages == 4)].shape[0] / user_ages.shape[0]

0.5827529275078729

### 用户性别分布分析

In [21]:
# 我们使用 value_counts 来计算用户信息中性别的分布信息。
df_user_info.gender.value_counts()

0.0    285634
1.0    121655
2.0     10419
Name: gender, dtype: int64

结合之前的数据集定义，0 代表女性，1 代表男性，2 代表未知。可以看到，阿普闪购的核心用户群是女性，是男性数量的 2.35 倍。

从用户群体的分析上，我们大概已经勾勒出我们的目标用户画像，25~34 岁之间的女性群体。但目前分析的只是注册用户的信息，**会不会存在男性虽然注册用户少但是购买力却更强呢**？为了验证这个假象，我们就需要结合订单数据来分析。

### user_log 与 user_info 表合并

分析完了独立的用户信息表后，我们希望分析不同用户群的下单行为特征。但是下单量在 user_log 表中，而用户信息在 user_info 表中，这样的话就不能简单地用 value_counts 来统计了，通过观察两个表的表结构不难发现，它们有共同的字段：user_id。

像这样的场景，我们可以通过 user_id 这样的共同字段来把两个表合并在一起。

In [22]:
df_user_log = df_user_log.join(df_user_info.set_index('user_id'), on='user_id')
df_user_log

,user_id,item_id,cat_id,seller_id,brand_id,date,action_type,timestamp,age_range,gender
0,328862,323294,833,2882,2661.0,829,click,62242.300959,6.0,1.0
1,328862,623866,1271,2882,2661.0,829,click,75598.841378,6.0,1.0
2,328862,239288,602,420,4953.0,801,click,69606.479645,6.0,1.0
3,328862,197239,602,420,4953.0,801,click,62758.912202,6.0,1.0
4,328862,524981,664,2382,1272.0,602,click,39651.123222,6.0,1.0
...,...,...,...,...,...,...,...,...,...,...
10985061,208016,869502,898,3716,5508.0,1110,click,58128.784724,NaN,NaN
10985062,208016,51712,898,3763,8434.0,1110,click,84907.294322,NaN,NaN
10985063,208016,952056,898,3763,8434.0,1110,click,64118.247166,NaN,NaN
10985064,208016,952056,898,3763,8434.0,1110,click,18877.391838,NaN,NaN


### 不同年龄段用户下单行为分析

首先我们需要过滤出下单用户，要过滤下单用户，集合数据集的定义，就是 action_type = order。 我们搭配 loc 函数与 value_counts 函数，即可实现针对下单用户的年龄段分析。

In [23]:
df_user_log.loc[df_user_log["action_type"] == "order", ["age_range"]].age_range.value_counts()

3.0    172525
4.0    153795
0.0    114908
5.0     79298
6.0     61534
2.0     59072
7.0     10785
8.0      1924
1.0        21
Name: age_range, dtype: int64

虽然 user_log 表中有很多行为，但这里我们核心关注的还是最终的转化：下单，所以我们筛选了行为为 “order”的记录，查看其年龄段分布。可以看到，下单的年龄段分布和用户信息的分布基本一致，25-34 岁的人占到 59.9%。

### 不同性别用户下单行为分析

In [25]:
# 接下来，我们进行下单用户的性别分析。
df_user_log.loc[df_user_log["action_type"] == "order", ["gender"]].gender.value_counts()

0.0    467381
1.0    161999
2.0     24482
Name: gender, dtype: int64

可以看到，女性用户不仅注册用户远超过男性，购买力同样惊人，是男性下单量的 2.9 倍。基本可以确定，我们发送营销信息要聚焦的用户群应该是 25～34 岁的女性。

### 不同日期的下单行为分析

虽然我们可以直接对 date 字段进行 value_count，但我们更希望是找到一个适合投放的**日期范围**，而不是具体某一天。所以这里我们使用分组的 value_count, 下面的代码我们将所有日期分成六组，统计每组的订单量的分布。

对 value_counts 的结果进行分组，可以通过给 value_counts 函数增加 bins 参数，bins 的值代表要分成几组，之后的数据分布就会按组输出。在这里，因为数据集是半年的，所以我们分六组（看每个月的分布）。

In [27]:
df_user_log.loc[df_user_log["action_type"] == "order", ["date"]].date.value_counts(bins=6)

(1011.0, 1111.0]    333721
(811.0, 911.0]       70699
(911.0, 1011.0]      69427
(510.399, 611.0]     68776
(611.0, 711.0]       62901
(711.0, 811.0]       54053
Name: date, dtype: int64

可以看到，用户在 10月中下旬一直到 11 月上旬这个时间段，下单量较为集中。分析完了日期分布后，接下来我们分析一下一天中的时间段的分布。

### 不同时间段的下单行为分析

timestamp 字段存储了每条记录下单的时间，从当天零点开始累积的秒数。并不是很直观，我们更希望可以基于小时级的数据去分析。所以我们考虑基于 timestamp 这一列，新创建一列时间，来表示小时。

新建列只需要直接给对应的列名赋值即可，值需要是一个合法的 Series。在这里，我们就是以 timestamp 列为基础，将其值除 3600， 然后用这个值创建一个新的列：time_hours_view.

In [32]:
df_user_log["time_hours_view"] = df_user_log["timestamp"]/3600
df_user_log

,user_id,item_id,cat_id,seller_id,brand_id,date,action_type,timestamp,age_range,gender,time_hours_view
0,328862.0,323294.0,833.0,2882.0,2661.0,829.0,click,62242.300959,6.0,1.0,17.289528
1,328862.0,623866.0,1271.0,2882.0,2661.0,829.0,click,75598.841378,6.0,1.0,20.999678
2,328862.0,239288.0,602.0,420.0,4953.0,801.0,click,69606.479645,6.0,1.0,19.335133
3,328862.0,197239.0,602.0,420.0,4953.0,801.0,click,62758.912202,6.0,1.0,17.433031
4,328862.0,524981.0,664.0,2382.0,1272.0,602.0,click,39651.123222,6.0,1.0,11.014201
...,...,...,...,...,...,...,...,...,...,...,...
10985063,208016.0,952056.0,898.0,3763.0,8434.0,1110.0,click,64118.247166,NaN,NaN,17.810624
10985064,208016.0,952056.0,898.0,3763.0,8434.0,1110.0,click,18877.391838,NaN,NaN,5.243720
10985065,208016.0,107662.0,898.0,1346.0,7995.0,1110.0,click,48918.482201,NaN,NaN,13.588467
time_hour_view,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


从输出可以看到，我们的 time_hours_view 已经被添加到了表格的最后一列。接下来的事情就比较简单了，我们直接用 value_count 来统计新增的 time_hours_view 字段，就可以实现对一天中的小时级分布进行分布统计。我们以两个小时为尺度，来查看分布，所以分为 12 组。

In [33]:
df_user_log.loc[df_user_log["action_type"] == "order", ["time_hours_view"]].time_hours_view.value_counts(bins=12)

(20.0, 22.0]     94209
(22.0, 24.0]     91529
(18.0, 20.0]     91330
(16.0, 18.0]     85681
(14.0, 16.0]     75372
(12.0, 14.0]     63580
(10.0, 12.0]     50909
(8.0, 10.0]      38938
(6.0, 8.0]       27962
(4.0, 6.0]       19428
(2.0, 4.0]       12639
(-0.025, 2.0]     8000
Name: time_hours_view, dtype: int64

从上述结果可以看到，晚上 8 点到 10 点之间是下单最为密集的，订单量为 94209 单。至此，我们本次短信营销的目标人群和时间就已经基本分析完毕了。我们应该针对 25～34 岁的女性，在 10 月中下旬到 11 月中旬的晚上 8 点到 10 点进行短信的批量发送，这样应该可以收获最好的转化效率。